In [1]:
import pandas as pd

In [2]:
df_Renewable = pd.read_csv('../Dataset/한국전력거래소_인천시_태양광_풍력_발전량_2024.csv', encoding='cp949')
df_climate = pd.read_csv('../Dataset/기후데이터_인천시_2024.csv', encoding='cp949')

In [3]:

df_Renewable["거래일자"] = pd.to_datetime(df_Renewable["거래일자"])

# 2) 시간(시) 정리: 1~24면 0~23으로 보정
h = df_Renewable["거래시간"].astype(int)

if h.min() == 1 and h.max() <= 24:
    h = h - 1  # 1->0시, 24->23시

# 3) 일시 생성 (datetime)
df_Renewable["일시"] = df_Renewable["거래일자"] + pd.to_timedelta(h, unit="h")

# 거래일자, 거래시간 열 제거후 일시 열을 맨 앞으로 이동
df_Renewable = df_Renewable.drop(columns=["거래일자", "거래시간"])
cols = ["일시"] + [col for col in df_Renewable.columns if col != "일시"]
df_Renewable = df_Renewable[cols]

In [4]:
df_Renewable

,일시,지역,연료원,전력거래량(MWh)
0,2024-01-01 00:00:00,인천시,태양광,0.000000
1,2024-01-01 01:00:00,인천시,태양광,0.000000
2,2024-01-01 02:00:00,인천시,태양광,0.000000
3,2024-01-01 03:00:00,인천시,태양광,0.000000
4,2024-01-01 04:00:00,인천시,태양광,0.000000
...,...,...,...,...
8779,2024-12-31 19:00:00,인천시,태양광,2.215681
8780,2024-12-31 20:00:00,인천시,태양광,1.068835
8781,2024-12-31 21:00:00,인천시,태양광,0.288480
8782,2024-12-31 22:00:00,인천시,태양광,0.000080


In [6]:
# df_climate 에서 QC플래그 열 제거, 지점 제거
df_climate = df_climate.drop(columns=[col for col in df_climate.columns if "QC플래그" in col])
df_climate = df_climate.drop(columns=["지점"])

In [10]:
# Fill NaN values with 0 in 일조(hr) and 일사(MJ/m2) 
df_climate["일조(hr)"] = df_climate["일조(hr)"].fillna(0)
df_climate["일사(MJ/m2)"] = df_climate["일사(MJ/m2)"].fillna(0)

In [11]:
# Fill missing values as 0 if needed
df_climate = df_climate.fillna(0)

In [12]:
df_climate

,지점명,일시,기온(°C),강수량(mm),풍속(m/s),풍향(16방위),습도(%),증기압(hPa),이슬점온도(°C),현지기압(hPa),...,운형(운형약어),최저운고(100m ),시정(10m),지면상태(지면상태코드),현상번호(국내식),지면온도(°C),5cm 지중온도(°C),10cm 지중온도(°C),20cm 지중온도(°C),30cm 지중온도(°C)
0,인천,2024-01-01 00:00,0.7,0.0,0.6,50.0,91.0,5.8,-0.6,1017.7,...,0,0.0,717.0,0.0,19.0,0.0,-0.7,0.2,1.0,1.8
1,인천,2024-01-01 01:00,0.6,0.0,1.1,70.0,92.0,5.9,-0.5,1018.1,...,0,0.0,606.0,0.0,19.0,-0.1,-0.7,0.2,1.0,1.7
2,인천,2024-01-01 02:00,0.3,0.0,1.8,360.0,93.0,5.8,-0.7,1018.5,...,0,13.0,306.0,0.0,19.0,-0.1,-0.7,0.3,0.9,1.7
3,인천,2024-01-01 03:00,0.5,0.0,0.7,320.0,93.0,5.9,-0.5,1019.0,...,Sc,9.0,479.0,0.0,19.0,-0.1,-0.7,0.3,1.0,1.8
4,인천,2024-01-01 04:00,0.4,0.0,1.5,70.0,94.0,5.9,-0.4,1019.2,...,Sc,10.0,381.0,0.0,19.0,-0.2,-0.7,0.3,1.0,1.8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8779,인천,2024-12-31 19:00,0.1,0.0,1.7,290.0,49.0,3.0,-9.3,1014.1,...,0,0.0,4861.0,0.0,0.0,0.1,1.3,2.2,2.6,3.1
8780,인천,2024-12-31 20:00,-0.1,0.0,2.2,270.0,53.0,3.2,-8.5,1014.1,...,0,0.0,4599.0,0.0,0.0,-0.2,1.1,2.0,2.5,3.2
8781,인천,2024-12-31 21:00,0.1,0.0,2.4,270.0,56.0,3.5,-7.6,1014.1,...,0,0.0,4925.0,0.0,0.0,-0.4,0.7,1.9,2.6,3.2
8782,인천,2024-12-31 22:00,0.3,0.0,2.3,290.0,59.0,3.7,-6.7,1014.3,...,0,0.0,4779.0,0.0,0.0,-0.5,0.8,1.7,2.5,3.2
